In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
import os

import cudf
import rmm

import nvtabular as nvt

In [3]:
from merlin_models.tensorflow.models.retrieval import YouTubeDNN

In [4]:
rmm.reinitialize(managed_memory=True)

In [5]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("./data/")
)
MODEL_BASE_DIR = os.environ.get(
    "MODEL_BASE_DIR", os.path.expanduser("./models/")
)
MODEL_LOG_DIR = os.environ.get(
    "MODEL_LOG_DIR", os.path.expanduser("./logs/")
)

In [6]:
examples = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "retrieval_training.parquet"))
examples.head()

search_terms  \
user_id day                                                       
1       4146  [40280, 34864, 60921, 55756, 53948, 968, 2649,...   
2       4071  [70987, 21602, 55885, 217, 26361, 361, 38094, ...   
3       7521  [63271, 34088, 581, 28225, 491, 34656, 41947, ...   
        7688  [69734, 68604, 71594, 56924, 32424, 31525, 668...   
        8045  [38644, 43776, 43440, 57466, 45557, 13968, 170...   

                                                         genres  \
user_id day                                                       
1       4146  [3, 10, 9, 17, 18, 3, 9, 19, 3, 4, 5, 6, 6, 9,...   
2       4071  [3, 10, 6, 16, 2, 3, 17, 18, 9, 17, 18, 2, 3, ...   
3       7521  [6, 9, 16, 19, 7, 12, 18, 3, 4, 5, 6, 10, 2, 1...   
        7688  [7, 9, 17, 18, 2, 9, 19, 6, 16, 8, 20, 6, 10, ...   
        8045  [7, 15, 18, 2, 18, 6, 7, 18, 4, 17, 6, 7, 9, 7...   

                                                     timestamps  \
user_id day                                                       
1       4146  [1147868053, 1147868097, 1147868414, 114786846...   
2       4071  [1141415528, 1141415566, 1141415576, 114141558...   
3       7521  [1439472199, 1439472203, 1439472215, 143947222...   
        7688  [1453904021, 1453904031, 1453904046, 145390404...   
        8045  [1484753654, 1484753762, 1484753766, 148475380...   

                                                      movie_ids  \
user_id day                                                       
1       4146  [5841, 1592, 1218, 6259, 3353, 1062, 6589, 384...   
2       4071  [5841, 493, 1339, 1592, 2550, 150, 234, 4781, ...   
3       7521  [352, 586, 1, 2481, 258, 315, 1167, 523, 12217...   
        7688  [1176, 1178, 10678, 9777, 11446, 11930, 10407,...   
        8045  [1063, 29365, 3908, 726, 763, 110, 213, 29375,...   

              movie_id_count  target_item  
user_id day                                
1       4146              38         7237  
2       4071             112         2061  
3       7521             247        24542  
        7688              17         4240  
        8045              30         9335

In [7]:
examples["search_terms"].dtype

ListDtype(int32)

In [8]:
examples["movie_id_count"].max()

5524

In [9]:
examples[["target_item", "timestamps"]]

target_item                                         timestamps
user_id day                                                                 
1       4146         7237  [1147868053, 1147868097, 1147868414, 114786846...
2       4071         2061  [1141415528, 1141415566, 1141415576, 114141558...
3       7521        24542  [1439472199, 1439472203, 1439472215, 143947222...
        7688         4240  [1453904021, 1453904031, 1453904046, 145390404...
        8045         9335  [1484753654, 1484753762, 1484753766, 148475380...
...                   ...                                                ...
162538  7513         6546  [1438780751, 1438780754, 1438780759, 143878083...
162539  2378         1615  [995149720, 995149760, 995149788, 995149788, 9...
162540  5315        11946  [1248854959, 1248855507, 1248855584, 124885572...
        5317        13355  [1249028189, 1249028584, 1249028593, 124902967...
162541  5223          290  [1240947363, 1240947373, 1240947377, 124094738...

[541872 rows x 2 columns]

## Hyper-parameters

In [10]:
BATCH_SIZE = 16  # Batch Size
CATEGORICAL_COLUMNS = []  # Single-hot
CATEGORICAL_MH_COLUMNS = ["search_terms", "movie_ids", "genres"]  # Multi-hot
NUMERIC_COLUMNS = ["movie_id_count"]

In [11]:
movie_workflow = nvt.Workflow.load(os.path.join(INPUT_DATA_DIR, "movie_features_workflow"))

In [12]:
EMBEDDING_TABLE_SHAPES, MH_EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(movie_workflow)
EMBEDDING_TABLE_SHAPES.update(MH_EMBEDDING_TABLE_SHAPES)
EMBEDDING_TABLE_SHAPES

{'movie_id': (62424, 512), 'tags_unique': (73051, 512), 'genres': (21, 16)}

In [13]:
EMBEDDING_TABLE_SHAPES["movie_ids"] = EMBEDDING_TABLE_SHAPES.pop('movie_id', None)
EMBEDDING_TABLE_SHAPES["search_terms"] = EMBEDDING_TABLE_SHAPES.pop('tags_unique', None)
EMBEDDING_TABLE_SHAPES

{'genres': (21, 16), 'movie_ids': (62424, 512), 'search_terms': (73051, 512)}

## DataLoader

In [14]:
import os
import tensorflow as tf

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# TF will have claimed all free GPU memory
os.environ["TF_MEMORY_ALLOCATION"] = "0.7"  # fraction of free memory
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater

In [15]:
train_dataset_tf = KerasSequenceLoader(
    os.path.join(INPUT_DATA_DIR, "retrieval_training.parquet"),  
    batch_size=BATCH_SIZE,
    label_names=["target_item"],
    cat_names=CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS,
    cont_names=NUMERIC_COLUMNS,
    engine="parquet",
    shuffle=True,
    buffer_size=0.25,
    parts_per_chunk=1,
)

In [16]:
batch = next(iter(train_dataset_tf))

In [17]:
continuous_cols = []

for col in NUMERIC_COLUMNS:
    continuous_cols.append(
        tf.feature_column.numeric_column(col)
    )
    
continuous_cols

[NumericColumn(key='movie_id_count', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [18]:
EMBEDDING_TABLE_SHAPES

{'genres': (21, 16), 'movie_ids': (62424, 512), 'search_terms': (73051, 512)}

In [19]:
embedding_dims = {}

for key, value in EMBEDDING_TABLE_SHAPES.items():
    embedding_dims[key] = 128  #value[1] # Latent dimensions

In [20]:
categorical_cols = []

for col in CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS:
    categorical_cols.append(
        tf.feature_column.categorical_column_with_identity(
            col, EMBEDDING_TABLE_SHAPES[col][0] # Cardinalities
        )
    )
    
categorical_cols

[IdentityCategoricalColumn(key='search_terms', number_buckets=73051, default_value=None),
 IdentityCategoricalColumn(key='movie_ids', number_buckets=62424, default_value=None),
 IdentityCategoricalColumn(key='genres', number_buckets=21, default_value=None)]

In [21]:
embedding_dims

{'genres': 128, 'movie_ids': 128, 'search_terms': 128}

In [22]:
model = YouTubeDNN(continuous_cols, categorical_cols, embedding_dims=embedding_dims, hidden_dims=[512,256,128])

In [23]:
model.input_layer.build({})
item_embeddings = model.input_layer.embedding_tables["movie_ids"]

def sampled_softmax_loss(y_true, y_pred):
    return tf.nn.sampled_softmax_loss(
        weights=item_embeddings,
        biases=tf.fill((item_embeddings.shape[0],), 0.01),
        labels=y_true,
        inputs=y_pred,
        num_sampled=20,
        num_classes=item_embeddings.shape[0],
    )

model.compile("nadam", sampled_softmax_loss)

In [24]:
# validation_callback = KerasSequenceValidater(valid_dataset_tf)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=MODEL_LOG_DIR)

history = model.fit(train_dataset_tf, callbacks=[tensorboard_callback], epochs=1)

33867/33867 [==============================] - 408s 11ms/step - loss: 2.2096


In [25]:
MODEL_NAME_TF = os.environ.get("MODEL_NAME_TF", "movielens_retrieval_tf")
MODEL_PATH_TEMP_TF = os.path.join(MODEL_BASE_DIR, MODEL_NAME_TF, "1/model.savedmodel")

# model.save(MODEL_PATH_TEMP_TF)
tf.keras.models.save_model(model, MODEL_PATH_TEMP_TF)

INFO:tensorflow:Assets written to: ./models/movielens_retrieval_tf/1/model.savedmodel/assets


In [26]:
rmm.reinitialize(managed_memory=False)

In [27]:
model.summary()

Model: "you_tube_dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  17343488  
_________________________________________________________________
dense (Dense)                multiple                  197632    
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dense_2 (Dense)              multiple                  32896     
Total params: 17,705,344
Trainable params: 17,705,344
Non-trainable params: 0
_________________________________________________________________


In [28]:
user_id = [[6]]
movie_id_count = [[1]]
movie_ids = [[882], [841], [1641], [523], [258], [315], [586], [601], [2767], [1167], [1174], [1183], [904], [1169], [892], [893], [1863], [899], [1071]]
genres = [[9], [16], [7], [9], [9], [16], [9], [19], [2], [3], [17], [7], [9], [7], [12], [18], [6], [7], [9], [18], [9], [16], [2], [3], [17], [9], [7], [9], [3], [9], [17], [2], [3], [9], [16], [11], [15], [9], [14], [3], [5], [10], [14], [5], [9], [17]]
search_terms = [[50389], [ 968], [10263], [8259], [20445], [58863], [18916], [66212], [63819], [9384], [45278], [20555], [39349], [18846], [39990], [53832], [2816], [62617], [42691]]

In [29]:
x = {
    "user_id": user_id,
    "movie_id_count": movie_id_count,
    "movie_ids": movie_ids,
    "genres": genres,
    "search_terms": search_terms,
}

for key, value in x.items():
    x[key] = tf.transpose(tf.convert_to_tensor(value))
    print(f"{key} shape: {x[key].shape}")

user_vector = model.predict(x)
user_vector

user_id shape: (1, 1)
movie_id_count shape: (1, 1)
movie_ids shape: (1, 19)
genres shape: (1, 46)
search_terms shape: (1, 19)


array([[ 0.12095062,  0.10068082, -0.17060989, -0.20062345,  0.1025757 ,
        -0.19021128,  0.09710072, -0.20167588,  0.10097501,  0.12444036,
        -0.20456296, -0.16610734, -0.17644118, -0.18693054,  0.19460878,
        -0.1721303 , -0.16572317, -0.1927955 ,  0.11263509, -0.21564102,
         0.10150526, -0.19281037,  0.10088726,  0.11158732, -0.23853041,
        -0.23801231,  0.11044632, -0.16581786,  0.1139873 ,  0.11684696,
        -0.18981439, -0.21554807,  0.1051307 ,  0.10400257, -0.20564163,
         0.09327978,  0.09857374, -0.2460519 , -0.1913882 , -0.22155826,
        -0.2093386 , -0.18920286, -0.16074476,  0.10172781, -0.22756658,
        -0.21138956, -0.19985506, -0.17381741, -0.18794313, -0.22774625,
         0.09118118, -0.25080186, -0.16943517,  0.13066807,  0.12236124,
         0.18873864, -0.15322898,  0.09767257, -0.23428485,  0.10778035,
        -0.18490016,  0.10718083, -0.21881583,  0.12091924,  0.1028012 ,
         0.09822867, -0.16372786, -0.16642824,  0.1

In [30]:
# Check that regularization was effective in learning dense (rather than sparse) representations
len(user_vector[user_vector == 0])

0